In [1]:
# Nash Cascade Neural Network
# A hydrologically intuitive deep learning network

# Set up a solution to a network of buckets where the number of buckets in each layer
# flows out to the buckets in the next layer
# The parameter on each bucket is the size and height of each spigot.

# Need a function that solves this individually at a single buckets
# Then a function that loops through and moves the water to the downstream buckets

In [2]:
import numpy as np
import torch
G = 9.81

In [3]:
# Initialize some flow into the system
U = [np.round(np.random.random()) for u in range(100)]
print(U)

[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0]


In [4]:
# Initialize the network size
Buckets_Per_Layer = [1,2,3,2,1] # Buckets Per Layer
print(len(Buckets_Per_Layer))

5


In [5]:
def get_initial_parameters_of_one_bucket(n_spigots):
    """
        returns tupple: {height, area, q}
    """
    s_parameters = [[np.random.random(),np.random.random()] for _ in range(n_spigots)]
    return s_parameters

In [6]:
def initialize_up_bucket_network(bpl):
    """Sets up the network of buckets
        Args: 
            bpl (list): the buckets per layer
        Returns:
            dict: A dictionary with all the buckets organized by layer, 
                  with each layer containing a dictionary, 
                  with keys for Head in the bucket and a list of spigot parms
    """

    # For each layer in the network, set up a dictionary with the bucket characteristics
    bucket_network_dict = {layer:{"H":[], "S":[], "s_q":[]} for layer in range(len(bpl))}
    
    for ilayer, n_buckets in enumerate(bpl):
        if ilayer < len(bpl)-1:
            n_spigots = bpl[ilayer+1]
        else:
            n_spigots = 1
        spigots = [get_initial_parameters_of_one_bucket(n_spigots) for jbucket in range(n_buckets)]
        bucket_network_dict[ilayer]["S"] = spigots
        bucket_network_dict[ilayer]["H"] = [100 for jbucket in range(n_buckets)]
        bucket_network_dict[ilayer]["s_q"] = [[0 for i in range(n_spigots)] for jbucket in range(n_buckets)]
        bucket_network_dict[ilayer]["theta"] = [[np.random.uniform(0.001, 0.01) for i in range(n_spigots)] for jbucket in range(n_buckets)]
    return bucket_network_dict

In [7]:
ncnn = initialize_up_bucket_network(Buckets_Per_Layer)

In [8]:
ncnn

{0: {'H': [100],
  'S': [[[0.3550534127236469, 0.2898577087602594],
    [0.08236047186249029, 0.8322673367299801]]],
  's_q': [[0, 0]],
  'theta': [[0.003084473233346086, 0.005433414573430011]]},
 1: {'H': [100, 100],
  'S': [[[0.1817810522039358, 0.9461166768861834],
    [0.12799485539759126, 0.6823147254997826],
    [0.460853682000506, 0.3791280828113539]],
   [[0.9199331710650406, 0.7905382515856079],
    [0.2159614699994411, 0.7220342552953847],
    [0.9292765773827322, 0.8414643527687837]]],
  's_q': [[0, 0, 0], [0, 0, 0]],
  'theta': [[0.003266964940267845, 0.003428210666356111, 0.006723872478769902],
   [0.0035479127892237604, 0.008617254522488444, 0.0077256410148742625]]},
 2: {'H': [100, 100, 100],
  'S': [[[0.2220501214945777, 0.010323763764213845],
    [0.7055737385193984, 0.8035202033138776]],
   [[0.8060290616129087, 0.3333909441703139],
    [0.7397123811560615, 0.7670756717984762]],
   [[0.4195100119664241, 0.30096328803311334],
    [0.4182047539372198, 0.7571172439447987

In [9]:
for ilayer in ncnn.keys():
    print("Network layer", ilayer)
    print("These are the characteristics of each spigot in a single bucket")
    print("Head", ncnn[ilayer]["H"])
    print("Spigot parms", ncnn[ilayer]["S"])

Network layer 0
These are the characteristics of each spigot in a single bucket
Head [100]
Spigot parms [[[0.3550534127236469, 0.2898577087602594], [0.08236047186249029, 0.8322673367299801]]]
Network layer 1
These are the characteristics of each spigot in a single bucket
Head [100, 100]
Spigot parms [[[0.1817810522039358, 0.9461166768861834], [0.12799485539759126, 0.6823147254997826], [0.460853682000506, 0.3791280828113539]], [[0.9199331710650406, 0.7905382515856079], [0.2159614699994411, 0.7220342552953847], [0.9292765773827322, 0.8414643527687837]]]
Network layer 2
These are the characteristics of each spigot in a single bucket
Head [100, 100, 100]
Spigot parms [[[0.2220501214945777, 0.010323763764213845], [0.7055737385193984, 0.8035202033138776]], [[0.8060290616129087, 0.3333909441703139], [0.7397123811560615, 0.7670756717984762]], [[0.4195100119664241, 0.30096328803311334], [0.4182047539372198, 0.7571172439447987]]]
Network layer 3
These are the characteristics of each spigot in a 

In [10]:
ncnn[1]["S"][0][0][-1]

0.9461166768861834

In [11]:
def solve_single_bucket(H, S, theta):
    """ solves a single bucket
        Args: 
            H (float): the water level in bucket
            S (list): the characteristics of the bucket outlets
                                        spigots[0]: Height of spigots, 
                                        spigots[1]: size of spigots
            theta (list): These are the weights of each bucket outlet (valve)
        Returns:
            H (float): the new water level in bucket
            s_q (list): The flow of water out of each spigot
    """
    print(f"This bucket has {len(S)} spigots, will return {len(S)} flows")
    s_q = []
    for S_i, theta_i in zip(S, theta):
        sp_h, sp_a = S_i
        h = np.max([0, H - sp_h])
        v =  theta_i * np.sqrt(2 * G * h)
        flow_out = v * sp_a
        s_q.append(flow_out)
    Q = np.sum(s_q)
    H = H - Q
    
    return H, s_q

In [12]:
for ilayer in list(ncnn.keys()):
    for ibucket in range(len(ncnn[ilayer]["H"])):
        
        print("S", ncnn[ilayer]["S"][ibucket])
        print("H before: ", ncnn[ilayer]["H"][ibucket])
        print("s_q before: ", ncnn[ilayer]["s_q"][ibucket])

        H = ncnn[ilayer]["H"][ibucket]
        S = ncnn[ilayer]["S"][ibucket]
        theta = ncnn[ilayer]["theta"][ibucket]

        H, s_q = solve_single_bucket(H, S, theta)
        
        ncnn[ilayer]["H"][ibucket] = H
        ncnn[ilayer]["s_q"][ibucket] = s_q

        print("H after: ", ncnn[ilayer]["H"][ibucket])
        print("s_q after: ", ncnn[ilayer]["s_q"][ibucket])

S [[0.3550534127236469, 0.2898577087602594], [0.08236047186249029, 0.8322673367299801]]
H before:  100
s_q before:  [0, 0]
This bucket has 2 spigots, will return 2 flows
H after:  99.76024907005299
s_q after:  [0.03953147341382072, 0.20021945653318415]
S [[0.1817810522039358, 0.9461166768861834], [0.12799485539759126, 0.6823147254997826], [0.460853682000506, 0.3791280828113539]]
H before:  100
s_q before:  [0, 0, 0]
This bucket has 3 spigots, will return 3 flows
H after:  99.64701433764303
s_q after:  [0.1367866083651405, 0.10354368863608657, 0.11265536535573879]
S [[0.9199331710650406, 0.7905382515856079], [0.2159614699994411, 0.7220342552953847], [0.9292765773827322, 0.8414643527687837]]
H before:  100
s_q before:  [0, 0, 0]
This bucket has 3 spigots, will return 3 flows
H after:  99.31442631221623
s_q after:  [0.12366262804735623, 0.27530034956609317, 0.2866107101703277]
S [[0.2220501214945777, 0.010323763764213845], [0.7055737385193984, 0.8035202033138776]]
H before:  100
s_q befor